In [1]:
# 超参设置
pretrained_path = r"D:\allApplications\forPython\llm\bert-base-chinese"
max_epoch_num = 1
output_dim = 2

In [16]:
import numpy as np

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

import datasets, evaluate  # 还需要安装accelerate

In [3]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_path)
model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_path, num_labels=output_dim
)

Some weights of the model checkpoint at D:\allApplications\forPython\llm\bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initial

In [ ]:
#import datasets
#dataset=datasets.load_dataset("Paul/hatecheck-mandarin")

#dataset.save_to_disk("hatecheck-mandarin")

In [4]:
all_dataset = datasets.load_from_disk("hatecheck-mandarin")

In [5]:
all_dataset["test"]

label_map = {"hateful": 1, "non-hateful": 0}


def tokenize_function(examples):
    return_dict = tokenizer(
        examples["test_case"], padding="max_length", truncation=True, max_length=512
    )
    return_dict["label"] = [label_map[x] for x in examples["label_gold"]]
    return return_dict


tokenized_datasets = all_dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at hatecheck-mandarin/test\cache-73c2a2dbf6047e28.arrow


In [6]:
tokenized_datasets["test"][1130].keys()

dict_keys(['mhc_case_id', 'functionality', 'test_case', 'label_gold', 'target_ident', 'ref_case_id', 'ref_templ_id', 'templ_id', 'case_templ', 'label_annotated', 'label_annotated_maj', 'disagreement_in_case', 'disagreement_in_template', 'input_ids', 'token_type_ids', 'attention_mask', 'label'])

In [7]:
print(tokenized_datasets["test"][1130]["test_case"])
print(tokenized_datasets["test"][1130]["label_gold"])
print(tokenized_datasets["test"][1130]["input_ids"])
print(tokenized_datasets["test"][1130]["token_type_ids"])
print(tokenized_datasets["test"][1130]["attention_mask"])
print(tokenized_datasets["test"][1130]["label"])

今天他妈的绝了。
non-hateful
[101, 791, 1921, 800, 1968, 4638, 5318, 749, 511, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [8]:
example_train_dataset = tokenized_datasets["test"].select(range(10))
example_valid_dataset = tokenized_datasets["test"].select(range(10, 20))
example_test_dataset = tokenized_datasets["test"].select(range(20, 30))

In [12]:
training_args = TrainingArguments(
    output_dir="test_checkpoint",
    evaluation_strategy="epoch",
    report_to="none",
    push_to_hub=False,
    num_train_epochs=max_epoch_num,
    per_device_train_batch_size=1,
    no_cuda=True,
    do_train=True,
    save_steps=10,
    save_total_limit=1,
)

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=example_train_dataset,
    eval_dataset=example_valid_dataset,
    compute_metrics=compute_metrics,
)

In [13]:
trainer.train()

d:\allApplications\forPython\anaconda3\lib\site-packages\transformers\optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.0042614201083779335, 'eval_accuracy': 1.0, 'eval_runtime': 18.1679, 'eval_samples_per_second': 0.55, 'eval_steps_per_second': 0.11, 'epoch': 1.0}
{'train_runtime': 63.9462, 'train_samples_per_second': 0.156, 'train_steps_per_second': 0.156, 'train_loss': 0.01458328366279602, 'epoch': 1.0}


TrainOutput(global_step=10, training_loss=0.01458328366279602, metrics={'train_runtime': 63.9462, 'train_samples_per_second': 0.156, 'train_steps_per_second': 0.156, 'train_loss': 0.01458328366279602, 'epoch': 1.0})

In [14]:
# 调用训练好的模型进行测试
result = trainer.predict(example_test_dataset)
print(result)

  0%|          | 0/2 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[-2.42961  ,  3.2694852],
       [-2.601208 ,  3.2841954],
       [-2.4568973,  3.0550063],
       [-2.48325  ,  3.239044 ],
       [-2.5793636,  3.283456 ],
       [-2.356497 ,  2.8706086],
       [-2.5480254,  3.2449925],
       [-2.4836488,  3.2778778],
       [-2.6639569,  3.2867384],
       [-2.5087523,  3.067698 ]], dtype=float32), label_ids=array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64), metrics={'test_loss': 0.003417523577809334, 'test_accuracy': 1.0, 'test_runtime': 17.7476, 'test_samples_per_second': 0.563, 'test_steps_per_second': 0.113})
